# Imports

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from skimage import io
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from PredictLetter import mp_detection, formatPoints

In [3]:
#### create array to signify the actions for iterating

# actions to be detected by model
actions = np.array(['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

# Preprocess Data and create labels

In [4]:
#### create all the path variables

# path for exported data (numpy arrays)
DATA_PATH = os.path.join('MP_DATA')

# the path for the data used to train our model
TRAIN_PATH = os.path.join(DATA_PATH, 'asl_alphabet_train')

# the path for the data used to test our model
TEST_PATH = os.path.join(DATA_PATH, 'asl_alphabet_test')

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
sequences, labels = [], []

# the number of files we are loading in from our converted numpy dataset to train out model
numTrainFiles = 4000
for action in actions:
    count = 0 
    for filename in os.listdir(os.path.join(TRAIN_PATH, action)):
        window = []
        keypoints = np.load(os.path.join(TRAIN_PATH, action, filename))
        window.append(keypoints)
        sequences.append(window)
        labels.append(label_map[action])
        count += 1
        if numTrainFiles == count:
            print(f"Loading Sequences and labels: '{action}' complete")
            break

Loading Sequences and labels: 'A' complete
Loading Sequences and labels: 'B' complete
Loading Sequences and labels: 'C' complete
Loading Sequences and labels: 'D' complete
Loading Sequences and labels: 'del' complete
Loading Sequences and labels: 'E' complete
Loading Sequences and labels: 'F' complete
Loading Sequences and labels: 'G' complete
Loading Sequences and labels: 'H' complete
Loading Sequences and labels: 'I' complete
Loading Sequences and labels: 'J' complete
Loading Sequences and labels: 'K' complete
Loading Sequences and labels: 'L' complete
Loading Sequences and labels: 'M' complete
Loading Sequences and labels: 'nothing' complete
Loading Sequences and labels: 'O' complete
Loading Sequences and labels: 'Q' complete
Loading Sequences and labels: 'R' complete
Loading Sequences and labels: 'S' complete
Loading Sequences and labels: 'space' complete
Loading Sequences and labels: 'U' complete
Loading Sequences and labels: 'V' complete
Loading Sequences and labels: 'W' complete

In [7]:
# the number of files we are choosing to train the model at the moment
# this should match our length of sequences
numTrainFiles*(len(actions))

116000

In [9]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [10]:
print(f"This should be {len(sequences)} if numTrainFiles is {numTrainFiles}: {len(y)}")
print(f"This should be 29 always: {len(y[0])}")

This should be 113665 if numTrainFiles is 4000: 113665
This should be 29 always: 29


# Splitting our data into training and testing partitions for our model training

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

# Build and Train LSTM Neural Network

In [12]:
# setting up the console logs to track info while training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# creating and building the model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1,126)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [13]:
# Here we train the model
model.fit(X_train, y_train,epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


3375/3375 [==============================] - 27s 5ms/step - loss: 1.6789 - categorical_accuracy: 0.4616
Epoch 2/2000
3375/3375 [==============================] - 17s 5ms/step - loss: 0.9341 - categorical_accuracy: 0.7008
Epoch 3/2000
3375/3375 [==============================] - 18s 5ms/step - loss: 0.7179 - categorical_accuracy: 0.7765
Epoch 4/2000
3375/3375 [==============================] - 17s 5ms/step - loss: 0.6073 - categorical_accuracy: 0.8089
Epoch 5/2000
3375/3375 [==============================] - 17s 5ms/step - loss: 0.5285 - categorical_accuracy: 0.8347
Epoch 6/2000
3375/3375 [==============================] - 17s 5ms/step - loss: 0.4546 - categorical_accuracy: 0.8578
Epoch 7/2000
3375/3375 [==============================] - 17s 5ms/step - loss: 0.3995 - categorical_accuracy: 0.8767
Epoch 8/2000
3375/3375 [==============================] - 17s 5ms/step - loss: 0.3532 - categorical_accuracy: 0.8906
Epoch 9/2000
3375/3375 [==============================] - 17s 

KeyboardInterrupt: 

In [16]:
# Here we save the model to be used later 
model.save("asl_Alphabet_Model_v.0.1.h5")

# Evaluation using confusion matrix and accuracy

In [18]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

178/178 [==============================] - 1s 3ms/step


In [19]:
# matrix representation for checking model accuracy
multilabel_confusion_matrix(ytrue,yhat)

array([[[5465,   11],
        [   7,  201]],

       [[5479,    2],
        [   1,  202]],

       [[5482,   11],
        [   2,  189]],

       [[5492,    2],
        [   2,  188]],

       [[5477,    8],
        [   7,  192]],

       [[5453,    5],
        [   6,  220]],

       [[5486,    3],
        [   2,  193]],

       [[5447,   12],
        [  15,  210]],

       [[5474,   14],
        [  13,  183]],

       [[5469,    4],
        [  13,  198]],

       [[5472,    7],
        [   7,  198]],

       [[5488,    7],
        [   5,  184]],

       [[5500,    1],
        [   1,  182]],

       [[5489,    5],
        [  10,  180]],

       [[5514,    6],
        [   8,  156]],

       [[5493,    0],
        [   0,  191]],

       [[5489,    5],
        [   4,  186]],

       [[5526,   10],
        [  14,  134]],

       [[5466,   10],
        [   9,  199]],

       [[5478,    5],
        [   0,  201]],

       [[5476,   13],
        [   4,  191]],

       [[5484,    6],
        [   

In [20]:
# percentage accuracy
accuracy_score(ytrue,yhat)

0.9692118226600985